<a href="https://colab.research.google.com/github/kwang2424/idiom_classifier/blob/master/bert_idiom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!wget https://github.com/zzeng13/DISC/raw/master/data/MAGPIE/processed/MAGPIE_random_read_comp_glove_embed.npy
!wget https://github.com/zzeng13/DISC/raw/master/data/MAGPIE/processed/MAGPIE_random_read_comp_glove_vocab.json

--2024-05-14 18:02:14--  https://github.com/zzeng13/DISC/raw/master/data/MAGPIE/processed/MAGPIE_random_read_comp_glove_embed.npy
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zzeng13/DISC/master/data/MAGPIE/processed/MAGPIE_random_read_comp_glove_embed.npy [following]
--2024-05-14 18:02:15--  https://raw.githubusercontent.com/zzeng13/DISC/master/data/MAGPIE/processed/MAGPIE_random_read_comp_glove_embed.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 92793728 (88M) [application/octet-stream]
Saving to: ‘MAGPIE_random_read_comp_glove_embed.npy’

MAGPIE_random_read_ 100%[===================>

In [ ]:
import numpy as np
glove_npy = np.load('MAGPIE_random_read_comp_glove_embed.npy')

In [ ]:
glove_npy.shape

(38664, 300)

In [ ]:
import json
glove_new = json.load(open('MAGPIE_random_read_comp_glove_vocab.json', 'r'))

In [ ]:
len(glove_new)

38664

In [ ]:
glove_vocab, glove_embeddings = [], []
for k in glove_new:
  n = int(k)
  glove_vocab.append(glove_new[k])
  glove_embeddings.append(glove_npy[n])

In [ ]:
# glove_vocab.append('<pad>')
# glove_embeddings.append([0] * 300)
glove_embeddings[2]
# glove_vocab[2]

array([ 3.74929994e-01, -6.52459979e-01,  9.11979973e-01,  6.61660016e-01,
       -4.49680001e-01, -3.34219992e-01, -1.50299996e-01,  2.65969992e-01,
       -1.27839994e+00, -1.88689995e+00, -1.54170007e-01, -7.09410012e-01,
        8.67969990e-01, -1.21900000e-01,  8.94219995e-01,  9.06050026e-01,
       -2.25930005e-01, -9.82410014e-01,  6.25129998e-01,  3.77220005e-01,
       -8.75999987e-01, -1.79250002e-01,  7.00600028e-01,  6.50340021e-01,
       -5.17570019e-01, -1.65040001e-01, -2.10630000e-01,  5.75219989e-01,
       -3.76969993e-01,  1.04639995e+00, -4.31450009e-01,  2.15990003e-02,
        8.93469974e-02, -1.84359998e-01, -4.66800004e-01, -1.06650002e-01,
       -6.36039972e-01,  9.90009978e-02, -7.17189983e-02,  8.05570006e-01,
        2.44859993e-01,  1.25819996e-01, -3.96970004e-01,  1.03710003e-01,
       -8.46840024e-01, -3.92560005e-01,  6.79520011e-01, -9.39349979e-02,
        1.61310002e-01, -1.50370002e-01,  1.10909998e-01,  4.13300008e-01,
        1.26650000e+00,  

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn as nn
from torch.optim import AdamW
import datasets

In [ ]:
glove_embeddings = torch.tensor(glove_embeddings, dtype=torch.float)
# embedding_layer = nn.Embedding.from_pretrained(glove_embeddings)

<ipython-input-10-5d2c2fc66683>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  glove_embeddings = torch.tensor(glove_embeddings, dtype=torch.float)


In [ ]:
word_to_index = { word: i for i, word in enumerate(glove_vocab) }

In [ ]:
hidden_lstm_size = 256
glove_dim = 300
bert_hidden_size = 768

In [ ]:
word_to_index['<UNK>']

1

In [ ]:
class ContextGloveEmbeddings(nn.Module):
  def __init__(self):
    super(ContextGloveEmbeddings, self).__init__()
    self.lstm = nn.LSTM(glove_dim, hidden_lstm_size, bidirectional=True, batch_first=True, dropout=0.2)

  def forward(self, input):
    h_n, c_n = self.lstm(input)
    return h_n

class ContextBertEmbeddings(nn.Module):
  def __init__(self):
    super(ContextBertEmbeddings, self).__init__()
    self.lstm = nn.LSTM(bert_hidden_size, hidden_lstm_size, bidirectional=True, batch_first=True, dropout=0.2)

  def forward(self, input):
    h_n, c_n = self.lstm(input)
    return h_n

In [ ]:
import torch.nn.functional as func

class AttentionLayer(nn.Module):
  def __init__(self):
    super(AttentionLayer, self).__init__()
    self.linear = nn.Linear(6*hidden_lstm_size, 1, bias=False)

  def forward(self, context, query):
    context_len, query_len = context.size(1), query.size(1)
    shape = (context.shape[0], context_len, query_len, hidden_lstm_size*2)  # (N, T, J, 2*h_dim)

    #similarity matrix
    context_ex = context.unsqueeze(2)           # (N, T, 1, 2*h_dim)
    context_ex = context_ex.expand(shape)  # (N, T, J, 2*h_dim)
    query_ex = query.unsqueeze(1)               # (N, 1, J, 2*h_dim)
    query_ex = query_ex.expand(shape)      # (N, T, J, 2*h_dim)

    combined = torch.cat((context_ex, query_ex, torch.mul(context_ex, query_ex)), 3)  # (N, T, J, 6*h_dim), [h;u;h◦u]
    S = self.linear(combined).view(context.shape[0], context_len, query_len)

    # Context2Query attention
    c2q_attn = torch.bmm(func.softmax(S, dim=-1), query)  # (N, T, h_dim) = bmm( (N, T, J), (N, J, h_dim) )

    # Query2Context attention
    b = func.softmax(torch.max(S, 2)[0], dim=-1)  # (N, T)
    q2c_attn = torch.bmm(b.unsqueeze(1), context)  # (N, 1, 2d) = bmm( (N, 1, T), (N, T, h_dim) )
    q2c_attn = q2c_attn.repeat(1, context_len, 1)  # (N, T, h_dim), tiled T times

    # combined attention: query aware representation of each context word
    G = torch.cat((context, c2q_attn, context.mul(c2q_attn), context.mul(q2c_attn)), 2)  # (N, T, 4*h_dim)
    return G

In [ ]:
class ModelLayer(nn.Module):
    def __init__(self):
      super(ModelLayer, self).__init__()
      self.lstm = nn.LSTM(hidden_lstm_size * 8, hidden_lstm_size,
                          bidirectional=True, batch_first=True, dropout=0.2)

    def forward(self, input):
        h_n, c_n = self.lstm(input)
        return h_n

In [ ]:
class TestModel(nn.Module):
  def __init__(self):
    super(TestModel, self).__init__()
    hidden_dim = 128
    vocab_size, glove_dim = glove_embeddings.shape
    print(vocab_size, glove_dim)

    self.bert = BertModel.from_pretrained('bert-base-uncased')
    # dimenions = [batch_size, seq_len, hidden_size (768)]
    self.bert_hidden_size = self.bert.config.hidden_size
    # dimensions = [40001, 300]
    self.glove_layer = nn.Embedding(vocab_size, glove_dim)
    self.glove_layer.weight = nn.Parameter(glove_embeddings)
    self.context_bert = ContextBertEmbeddings()
    self.context_glove = ContextGloveEmbeddings()
    self.attention = AttentionLayer()
    self.model = ModelLayer()
    self.output = nn.Linear(hidden_lstm_size*2, 5)

  def forward(self, input_ids, attention_mask, glove_idx):
    bert_embeddings = self.bert(input_ids=input_ids,
                                attention_mask=attention_mask).last_hidden_state
    glove_embeddings = self.glove_layer(glove_idx)
    bert_lstm = self.context_bert(bert_embeddings)
    glove_lstm = self.context_glove(glove_embeddings)
    attn = self.attention(bert_lstm, glove_lstm)
    model = self.model(attn)
    output = self.output(model)
    return func.softmax(output, dim=-1)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

y_train = """0 1 2 2 3 4"""
y_split = y_train.split(' ')
y_train_arr = []
for i in y_split:
  y_train_arr.append(int(i))

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train_arr), y=y_train_arr)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


In [ ]:
model = TestModel()
model.load_state_dict(torch.load('model_weights1.pth'))
model.to(device)
# model.load_state_dict(torch.load('model_weights.pth'))
# print(model)
# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
loss = nn.CrossEntropyLoss(weight=class_weights)

38664 300


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
class TokenizedTextDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        tokens = row['tokens']
        label = row['tags']

        if len(label) < 256:
          padding = [0] * (256 - len(label))
          label = label + padding
        else:
          label = label[:256]
        glove_idx = []
        for token in tokens:
          lower_token = token.lower()
          if lower_token not in word_to_index:
            lower_token = '<UNK>'
          glove_idx.append(word_to_index[lower_token])


        text = ' '.join(tokens)

        inputs = self.tokenizer(
            text,
            padding="max_length",
            max_length=256,
            truncation=True,
            return_tensors='pt'
        )
        # print(glove_idx, len(glove_idx))

        if len(glove_idx) < 256:
          padding = [word_to_index['<PAD>']] * (256 - len(glove_idx))
          glove_idx = glove_idx + padding
        elif len(glove_idx) > 256:
          glove_idx = glove_idx[:256]
        # print(glove_idx, '\n', len(glove_idx))
        inputs['glove_idx'] = torch.tensor(glove_idx, dtype=torch.long)

        inputs['label'] = torch.tensor(label, dtype=torch.long)
        return inputs

In [ ]:
data = datasets.load_dataset('Gooogr/pie_idioms', split="train[:50%]")
train_testvalid = data.train_test_split(test_size=0.1)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
train_iter = train_testvalid['train']

Generating train split:   0%|          | 0/46090 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5761 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5762 [00:00<?, ? examples/s]

In [ ]:
train_tokens = []
train_tags = []
for x in train_iter:
  new_tags = [1]
  tags = [3] + [2 if tag > 0 else 1 for tag in x['ner_tags']]
  tags.append(4)
  new_arr = ['<S>']
  new_arr += x['tokens']
  new_arr.append('<E>')
  train_tokens.append(new_arr)
  train_tags.append(tags)

test_tokens = []
test_tags = []
for x in test_valid['test']:
  new_tags = [1]
  tags = [3] + [2 if tag > 0 else 1 for tag in x['ner_tags']]
  tags.append(4)
  new_arr = ['<S>']
  new_arr += x['tokens']
  new_arr.append('<E>')
  test_tokens.append(new_arr)
  test_tags.append(tags)

valid_tokens = []
valid_tags = []
for x in test_valid['train']:
  new_tags = [1]
  tags = [3] + [2 if tag > 0 else 1 for tag in x['ner_tags']]
  tags.append(4)
  new_arr = ['<S>']
  new_arr += x['tokens']
  new_arr.append('<E>')
  valid_tokens.append(new_arr)
  valid_tags.append(tags)

In [ ]:
tokens = [x['tokens'] for x in train_iter]
is_pie = [1 if x['is_pie'] else 0 for x in train_iter]
ner_tags = [[2 if tag > 0 else 1 for tag in tags] for tags in train_iter['ner_tags']]

data = pd.DataFrame({
    "tokens": train_tokens,
    "is_pie": is_pie,
    "tags": train_tags
})

# print(test_valid)
# test_tokens = [x['tokens'] for x in test_valid['test']]
test_is_pie = [1 if x['is_pie'] else 0 for x in test_valid['test']]
# test_tags = [[2 if tag > 0 else 1 for tag in tags] for tags in test_valid['test']['ner_tags']]
test_data = pd.DataFrame({
    "tokens": test_tokens,
    "is_pie": test_is_pie,
    "tags": test_tags
})

# valid_tokens = [x['tokens'] for x in test_valid['train']]
valid_is_pie = [1 if x['is_pie'] else 0 for x in test_valid['train']]
# valid_tags = [[2 if tag > 0 else 1 for tag in tags] for tags in test_valid['train']['ner_tags']]
valid_data = pd.DataFrame({
    "tokens": valid_tokens,
    "is_pie": valid_is_pie,
    "tags": valid_tags
})

# Create the dataset and dataloader
dataset = TokenizedTextDataset(data)
test_dataset = TokenizedTextDataset(test_data)
valid_dataset = TokenizedTextDataset(valid_data)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
torch.set_printoptions(profile="full")

In [ ]:
model.eval()  # Set the model to evaluation mode
all_preds = []
all_labels = []
i = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['label'].to(device)
        # print('labels', labels)
        glove_idx = batch['glove_idx'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        glove_idx=glove_idx)
        # print(outputs)
        preds = torch.argmax(outputs, dim=2)  # Ensure this matches your output dimension
        # print(preds)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())
        i += 1
        if i == 11:
          break

# Flatten lists if necessary
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)
i = 0
for i in range(15):
  print('preds:', all_preds[i])
  print('labels:', all_labels[i])
  print('\n')

preds: [3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
labels: [3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2
 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 

In [ ]:
# {"<PAD>": 0, "<l>": 1, "<i>": 2, "<s>": 3, "<e>": 4}
new_preds = []
new_labels = []
for i, label in enumerate(all_labels):
  curr_pred, curr_label = [], []
  pred = all_preds[i]
  end = 0
  for j, val in enumerate(label):
    if val == 2:
      curr_label.append(1)
    elif val == 0:
      end = j
      # print('here!', end, j, len(curr_label))
      break
    else:
      curr_label.append(0)
  if len(curr_label) != end:
    end = len(curr_label)
  for j, val in enumerate(pred):
    if j == end:
      break
    if val == 2:
      curr_pred.append(1)
    else:
      curr_pred.append(0)
  print(curr_pred)
  new_labels.append(curr_label)
  new_preds.append(curr_pred)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
total = len(new_labels)
accuracy = 0
precision = 0
recall = 0
f1 = 0
for i in range(len(new_labels)):
  # print(len(new_labels[i]), len(new_preds[i]))
  # print(new_labels[i], '\n', new_preds[i])
#   accuracy += accuracy_score(new_labels[i], new_preds[i])
#   precision += precision_score(new_labels[i], new_preds[i], average="binary", pos_label=1)  # Adjust average method if needed
#   recall = recall_score(new_labels[i], new_preds[i], average="micro", zero_division=1.0)
#   f1 = f1_score(new_labels[i], new_preds[i], average="micro", zero_division=1.0)
  accuracy += 1 if sum(np.equal(new_labels[i], new_preds[i])) == len(new_labels[i]) else 0
print(accuracy / total)
# print(precision / total)
# print(recall / total)
# print(f1 / total)

0.0


In [ ]:
i = 0
model.train()
total_loss = 0
for batch in train_loader:
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['label'].to(device)
        glove_idx = batch['glove_idx'].to(device)
        # print(glove_idx.shape)
        optimizer.zero_grad()  # Clear previous gradients

        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        glove_idx=glove_idx)

        print('output:', outputs, 'label: ', labels, '\n')
        # print(labels, labels.shape, labels.reshape(-1))
        ce_loss = loss(outputs.reshape(-1, 2), labels.reshape(-1))
        # print(labels.reshape(-1))
        # print(outputs.reshape(-1, 2))
        ce_loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model parameters

        total_loss += ce_loss.item()  # Accumulate loss
        i += 1
        if i == 10:
          break

In [ ]:
def train_epoch(model, dataloader, optimizer, device):
    model.train()  # Set the model to training mode
    total_loss = 0

    for batch in dataloader:
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        labels = batch['label'].to(device)
        glove_idx = batch['glove_idx'].to(device)
        # print(glove_idx.shape)
        optimizer.zero_grad()  # Clear previous gradients

        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        glove_idx=glove_idx)

        # print(outputs, outputs.shape, outputs.reshape(-1, 2))
        # print(labels, labels.shape, labels.reshape(-1))
        ce_loss = loss(outputs.reshape(-1, 5), labels.reshape(-1))

        ce_loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update model parameters

        total_loss += ce_loss.cpu().detach().item()  # Accumulate loss

    return total_loss / len(dataloader)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            labels = batch['label'].to(device)
            glove_idx = batch['glove_idx'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                            glove_idx=glove_idx)
            preds = torch.argmax(outputs, dim=2)  # Ensure this matches your output dimension
            # print(preds)
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())


    # Flatten lists if necessary
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    new_preds = []
    new_labels = []
    for i, label in enumerate(all_labels):
      curr_pred, curr_label = [], []
      pred = all_preds[i]
      end = 0
      for j, val in enumerate(label):
        if val > 0:
          curr_label.append(val-1)
        elif val == 0:
          end = j
          # print('here!', end, j, len(curr_label))
          break
      if len(curr_label) != end:
        end = len(curr_label)
      for j, val in enumerate(pred):
        if j == end:
          break
        if val > 0:
          curr_pred.append(val-1)
        else:
          curr_pred.append(0)
      # print(curr_pred)
      new_labels.append(curr_label)
      new_preds.append(curr_pred)
    # Compute metrics
    # print(all_preds, all_labels)
    total = len(new_labels)
    accuracy = 0
    precision = 0
    recall = 0
    f1 = 0
    cnt = 0
    for i in range(len(new_labels)):
      # print(len(new_labels[i]), len(new_preds[i]))
      # print(new_labels[i], '\n', new_preds[i])
      accuracy += accuracy_score(new_labels[i], new_preds[i])
      precision += precision_score(new_labels[i], new_preds[i], average="macro", zero_division=1.0)  # Adjust average method if needed
      recall += recall_score(new_labels[i], new_preds[i], average="macro", zero_division=1.0)
      f1 += f1_score(new_labels[i], new_preds[i], average="macro", zero_division=1.0)
      cnt += 1 if sum(np.equal(new_labels[i], new_preds[i])) == len(new_labels[i]) else 0
    print(cnt / total)

    return accuracy / total, precision / total , recall / total, f1 / total


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, device)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {train_loss}")

    # Evaluation
    accuracy, precision, recall, f1 = evaluate(model, train_loader, device)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

Epoch 1/5
Train Loss: 0.9160333736909203
0.0021697203471552555
Accuracy: 0.9628262437503421
Precision: 0.9903878118278973
Recall: 0.528752812600439
F1 Score: 0.523794027023564
Epoch 2/5
Train Loss: 0.9149407058712878
0.0021697203471552555
Accuracy: 0.962826489294825
Precision: 0.9903879228949863
Recall: 0.5287568306010824
F1 Score: 0.5237981012950481
Epoch 3/5
Train Loss: 0.9141099810186519
0.0849566055930569
Accuracy: 0.9688417086511173
Precision: 0.9640246391245348
Recall: 0.6682607707792054
F1 Score: 0.6644486634549769
Epoch 4/5
Train Loss: 0.9130106358007717
0.08881388621022179
Accuracy: 0.9689916537599732
Precision: 0.9530158604345333
Recall: 0.6794484334222894
F1 Score: 0.6756933774119919
Epoch 5/5
Train Loss: 0.9124505984429176
0.08717454194792672
Accuracy: 0.9689162279367244
Precision: 0.9493758232876588
Recall: 0.6721869112653054
F1 Score: 0.6684536899255051


In [ ]:
print("Validating dataset.")
valid_acc, valid_prec, valid_recall, valid_f1 = evaluate(model, valid_loader, device)
print(f"Accuracy: {valid_acc}")
print(f"Precision: {valid_prec}")
print(f"Recall: {valid_recall}")
print(f"F1 Score: {valid_f1}")

Validating dataset.
0.06597222222222222
Accuracy: 0.969007015743744
Precision: 0.9371225581264339
Recall: 0.648865152451028
F1 Score: 0.6452728091498947


In [ ]:
print("Seeing metrics on Test dataset")
test_acc, test_prec, test_recall, test_f1 = evaluate(model, test_loader, device)
print(f"Accuracy: {test_acc}")
print(f"Precision: {test_prec}")
print(f"Recall: {test_recall}")
print(f"F1 Score: {test_f1}")


Seeing metrics on Test dataset
0.0997398091934085
Accuracy: 0.9687120555145741
Precision: 0.9447736787654639
Recall: 0.6682172498294456
F1 Score: 0.6645209174525546


In [ ]:
torch.save(model, 'model.pth')

In [ ]:
torch.save(model.state_dict(), 'model_weights2.pth')